In [1]:
from os.path import join, dirname
from dotenv import load_dotenv
import os
import pickle
from snowflake import connector
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter, StrMethodFormatter
from scipy.optimize import curve_fit
from tpot import TPOTRegressor
from sklearn.model_selection import train_test_split, RepeatedKFold
from sklearn.metrics import r2_score
# from sklearn.preprocessing import Normalizer, QuantileTransformer, RobustScaler, PolynomialFeatures
# from sklearn.pipeline import Pipeline
# from sklearn.model_selection import GridSearchCV, train_test_split
# from sklearn.neighbors import KNeighborsRegressor
# from sklearn.linear_model import LinearRegression, Lasso, Ridge
# from sklearn.svm import SVR
# from sklearn.neural_network import MLPRegressor
# from sklearn.feature_selection import SelectKBest, f_classif
# from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_squared_log_error, mean_absolute_percentage_error, median_absolute_error, max_error, make_scorer


pd.options.display.float_format = '{:,.2f}'.format

# get environment variables
dotenv_path = join(dirname('streamlit_grs_fit\\app\\'), '.env')
load_dotenv(dotenv_path)
SF_ACCOUNT = os.getenv('SF_ACCOUNT')
SF_USER = os.getenv('SF_USER')
SF_PASSWORD = os.getenv('SF_PASSWORD')
SF_ROLE = os.getenv('SF_ROLE')
SF_WAREHOUSE = os.getenv('SF_WAREHOUSE')
SF_DATABASE = os.getenv('SF_DATABASE')
SF_SCHEMA = os.getenv('SF_SCHEMA')

def load_data(query):
    conn = connector.connect(
        user = SF_USER
        ,password = SF_PASSWORD
        ,account = SF_ACCOUNT
        ,warehouse = SF_WAREHOUSE
        ,database = SF_DATABASE
        ,schema = SF_SCHEMA
        ,role = SF_ROLE
    )
    cur = conn.cursor()
    df_data = cur.execute(query).fetch_pandas_all()
    return df_data

c:\Users\SwanS\Anaconda3\envs\grs_fit\lib\site-packages\tpot\builtins\__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


In [2]:
query = 'select '+\
            'JOB'+\
            ',DIRECT_COST'+\
            ',DIV_00_DIRECT_COST'+\
            ',DIV_01_DIRECT_COST'+\
            ',DIV_02_DIRECT_COST'+\
            ',DIV_03_DIRECT_COST'+\
            ',DIV_04_DIRECT_COST'+\
            ',DIV_05_DIRECT_COST'+\
            ',DIV_06_DIRECT_COST'+\
            ',DIV_07_DIRECT_COST'+\
            ',DIV_08_DIRECT_COST'+\
            ',DIV_09_DIRECT_COST'+\
            ',DIV_10_DIRECT_COST'+\
            ',DIV_11_DIRECT_COST'+\
            ',DIV_12_DIRECT_COST'+\
            ',DIV_13_DIRECT_COST'+\
            ',DIV_14_DIRECT_COST'+\
            ',DIV_15_DIRECT_COST'+\
            ',DIV_16_DIRECT_COST'+\
            ',DIV_17_DIRECT_COST'+\
            ',DIV_18_DIRECT_COST'+\
            ',DIV_19_DIRECT_COST'+\
            ',DIV_21_DIRECT_COST'+\
            ',DIV_22_DIRECT_COST'+\
            ',DIV_23_DIRECT_COST'+\
            ',DIV_26_DIRECT_COST'+\
            ',DIV_27_DIRECT_COST'+\
            ',DIV_28_DIRECT_COST'+\
            ',DIV_31_DIRECT_COST'+\
            ',DIV_32_DIRECT_COST'+\
            ',DIV_33_DIRECT_COST'+\
            ',DIV_34_DIRECT_COST'+\
            ',DIV_55_DIRECT_COST'+\
            ',GCS_COST'+\
            ',GRS_COST '+\
            'from sandbox.global.ml_grs_fit ' 
df_data = load_data(query).set_index('JOB') 
df_data = pd.DataFrame(df_data)
df_data = df_data.fillna(0)

In [3]:
df_working = df_data.loc[
                    (0 != df_data.GRS_COST) &
                    (0 != df_data.GCS_COST)
].copy()
df_working.describe()

,DIRECT_COST,DIV_00_DIRECT_COST,DIV_01_DIRECT_COST,DIV_02_DIRECT_COST,DIV_03_DIRECT_COST,DIV_04_DIRECT_COST,DIV_05_DIRECT_COST,DIV_06_DIRECT_COST,DIV_07_DIRECT_COST,DIV_08_DIRECT_COST,...,DIV_26_DIRECT_COST,DIV_27_DIRECT_COST,DIV_28_DIRECT_COST,DIV_31_DIRECT_COST,DIV_32_DIRECT_COST,DIV_33_DIRECT_COST,DIV_34_DIRECT_COST,DIV_55_DIRECT_COST,GCS_COST,GRS_COST
count,"3,332.00","3,332.00","3,332.00","3,332.00","3,332.00","3,332.00","3,332.00","3,332.00","3,332.00","3,332.00",...,"3,332.00","3,332.00","3,332.00","3,332.00","3,332.00","3,332.00","3,332.00","3,332.00","3,332.00","3,332.00"
mean,"2,753,305.18","15,416.14","1,095.30","120,305.74","305,087.19","25,975.15","192,784.27","83,875.74","97,562.86","262,156.35",...,"279,623.43","22,192.54","7,159.77","84,671.51","18,546.22","19,750.99",505.83,"1,710.05","159,975.26","130,762.61"
std,"18,205,353.59","247,803.94","21,856.71","835,518.53","2,456,075.21","255,679.11","1,926,522.10","479,208.15","712,596.43","2,347,850.24",...,"2,283,423.57","355,302.65","98,050.42","852,341.42","200,217.24","315,920.99","14,397.44","98,425.42","991,159.86","893,014.97"
min,"-3,404,153.00","-327,264.32",0.00,"-41,282.56","-4,315,458.20","-164,664.12","-1,398,362.17","-174,550.72","-16,197.24","-407,656.41",...,-104.77,0.00,0.00,"-2,322.10",-11.50,"-53,330.53",-4.20,0.00,"-3,335,087.00","-629,785.00"
25%,"4,410.75",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,554.25,305.75
50%,"31,937.00",0.00,0.00,889.22,0.00,0.00,0.00,353.55,0.00,37.68,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"4,763.00","2,099.00"
75%,"279,720.25",0.00,0.00,"10,486.36","1,041.34",0.00,180.50,"10,992.53",44.95,"9,230.26",...,"3,893.87",0.00,0.00,0.00,0.00,0.00,0.00,0.00,"35,369.50","16,783.00"
max,"399,570,438.00","8,618,727.16","762,087.69","26,715,320.64","49,560,386.64","9,635,075.36","51,850,542.49","9,711,287.61","19,337,427.69","56,887,683.67",...,"57,604,860.12","14,904,687.98","2,706,150.04","19,786,153.90","6,898,015.60","10,785,799.69","607,500.00","5,681,453.04","23,749,853.00","18,182,714.00"


In [24]:
X  = list(df_working.iloc[:,:-2].columns)
y_grs = list(df_working.iloc[:,-1:].columns)
y_gcs = list(df_working.iloc[:,-2:-1].columns)
y_gcs

['GCS_COST']

I've tried a number of estimators, and none has beat the k-nearest neighbor regressor. We're going to change it up a bit now, and use a genetic programming tool called TPOT and see how that does

In [25]:
X_train, X_test, y_grs_train, y_grs_test = train_test_split(df_working[X], df_working[y_grs].values.ravel(), test_size=0.2, random_state=42)
X_gcs_train, X_gcs_test, y_gcs_train, y_gcs_test = train_test_split(df_working[X], df_working[y_gcs].values.ravel(), test_size=0.2, random_state=42)

In [12]:
tpot = TPOTRegressor(generations=5, population_size=50, verbosity=2, random_state=42)
tpot.fit(X_train, y_grs_train)
print(tpot.score(X_test, y_grs_test))
tpot.export('tpot_grs_pipeline.py')

                                                                              
Generation 1 - Current best internal CV score: -169620157638.27933
                                                                              
Generation 2 - Current best internal CV score: -169620157638.27933
                                                                              
Generation 3 - Current best internal CV score: -161401803546.80942
                                                                              
Generation 4 - Current best internal CV score: -156260656385.51758
                                                                              
Generation 5 - Current best internal CV score: -156260656385.51758
                                                                              
Best pipeline: GradientBoostingRegressor(MinMaxScaler(input_matrix), alpha=0.75, learning_rate=0.1, loss=huber, max_depth=6, max_features=0.8, min_samples_leaf=1, min_samples_split=19, n_esti

In [26]:
tpot_gcs = TPOTRegressor(generations=5, population_size=50, verbosity=2, random_state=42)
tpot_gcs.fit(X_gcs_train, y_gcs_train)
print(tpot_gcs.score(X_gcs_test, y_gcs_test))
tpot.export('tpot_gcs_pipeline.py')

                                                                              
Generation 1 - Current best internal CV score: -454980662761.639
                                                                              
Generation 2 - Current best internal CV score: -448118718508.5226
                                                                              
Generation 3 - Current best internal CV score: -448118718508.5226
                                                                              
Generation 4 - Current best internal CV score: -448118718508.5226
                                                                              
Generation 5 - Current best internal CV score: -448118718508.5226
                                                                              
Best pipeline: XGBRegressor(ZeroCount(input_matrix), learning_rate=0.1, max_depth=4, min_child_weight=3, n_estimators=100, n_jobs=1, objective=reg:squarederror, subsample=0.45, verbosity=0)
-289694

In [ ]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)
tpot_75 = TPOTRegressor(generations=5, population_size=75, verbosity=2, cv=cv, random_state=42, n_jobs=-2)
tpot_75.fit(X_train, y_grs_train)
print(tpot_75.score(X_test, y_grs_test))
tpot_75.export('tpot_75_grs_pipeline.py')

In [27]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)
tpot_75_gcs = TPOTRegressor(generations=5, population_size=75, verbosity=2, cv=cv, random_state=42, n_jobs=-2)
tpot_75_gcs.fit(X_gcs_train, y_gcs_train)
print(tpot_75_gcs.score(X_gcs_test, y_gcs_test))
tpot.export('tpot_75_gcs_pipeline.py')

                                                                               
Generation 1 - Current best internal CV score: -466665305372.89484
                                                                                
Generation 2 - Current best internal CV score: -466665305372.89484
                                                                              
Generation 3 - Current best internal CV score: -466665305372.89484
                                                                              
Generation 4 - Current best internal CV score: -465474810426.29193
                                                                              
Generation 5 - Current best internal CV score: -461777905481.6639
                                                             
Best pipeline: RandomForestRegressor(RandomForestRegressor(input_matrix, bootstrap=True, max_features=0.55, min_samples_leaf=19, min_samples_split=14, n_estimators=100), bootstrap=False, max_features=0.1, m

c:\Users\SwanS\Anaconda3\envs\grs_fit\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [22]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)
tpot_75_r2 = TPOTRegressor(generations=5, population_size=75, scoring='r2', verbosity=2, cv=cv, random_state=42, n_jobs=-2)
tpot_75_r2.fit(X_train, y_grs_train)
print(tpot_75_r2.score(X_test, y_grs_test))
tpot_75_r2.export('tpot_75_r2_grs_pipeline.py')

                                                                               
Generation 1 - Current best internal CV score: 0.7567428441911429
                                                                                
Generation 2 - Current best internal CV score: 0.7567428441911429
                                                                              
Generation 3 - Current best internal CV score: 0.7567428441911429
                                                                              
Generation 4 - Current best internal CV score: 0.7647658556084654
                                                                              
Generation 5 - Current best internal CV score: 0.7647658556084654
                                                                              
Best pipeline: ElasticNetCV(XGBRegressor(input_matrix, learning_rate=0.1, max_depth=5, min_child_weight=10, n_estimators=100, n_jobs=1, objective=reg:squarederror, subsample=0.9500000000000001,

In [29]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)
tpot_75_r2_gcs = TPOTRegressor(generations=5, population_size=75, scoring='r2', verbosity=2, cv=cv, random_state=42, n_jobs=-2)
tpot_75_r2_gcs.fit(X_gcs_train, y_gcs_train)
print(tpot_75_r2_gcs.score(X_gcs_test, y_gcs_test))
tpot.export('tpot_75_r2_gcs_pipeline.py')

                                                                               
Generation 1 - Current best internal CV score: 0.5945191543603052
                                                                              
Generation 2 - Current best internal CV score: 0.5986198913067237
                                                                              
Generation 3 - Current best internal CV score: 0.6008664219379748
                                                                              
Generation 4 - Current best internal CV score: 0.6008664219379748
                                                                              
Generation 5 - Current best internal CV score: 0.6008664219379748
                                                           
Best pipeline: RandomForestRegressor(SelectFwe(input_matrix, alpha=0.037), bootstrap=True, max_features=0.15000000000000002, min_samples_leaf=2, min_samples_split=3, n_estimators=100)
0.8551568838841415


In [23]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)
tpot_75_negmae = TPOTRegressor(generations=5, population_size=75, scoring='neg_median_absolute_error', verbosity=2, cv=cv, random_state=42, n_jobs=-2)
tpot_75_negmae.fit(X_train, y_grs_train)
print(tpot_75_negmae.score(X_test, y_grs_test))
tpot_75_r2.export('tpot_75_negmae_grs_pipeline.py')

                                                                               
Generation 1 - Current best internal CV score: -2028.0356968607452
                                                                                
Generation 2 - Current best internal CV score: -1900.681771666977
                                                                              
Generation 3 - Current best internal CV score: -1900.681771666977
                                                                              
Generation 4 - Current best internal CV score: -1879.8812673367806
                                                                                
Generation 5 - Current best internal CV score: -1879.8812673367806
                                                             
Best pipeline: GradientBoostingRegressor(input_matrix, alpha=0.75, learning_rate=0.01, loss=huber, max_depth=5, max_features=0.55, min_samples_leaf=14, min_samples_split=17, n_estimators=100, subsample=0.3

In [30]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)
tpot_75_negmae_gcs = TPOTRegressor(generations=5, population_size=75, scoring='neg_median_absolute_error', verbosity=2, cv=cv, random_state=42, n_jobs=-2)
tpot_75_negmae_gcs.fit(X_gcs_train, y_gcs_train)
print(tpot_75_negmae_gcs.score(X_gcs_test, y_gcs_test))
tpot.export('tpot_75_negmae_gcs_pipeline.py')

                                                                               
Generation 1 - Current best internal CV score: -3682.918929821638
                                                                              
Generation 2 - Current best internal CV score: -3682.9075127652673
                                                                              
Generation 3 - Current best internal CV score: -3682.9075127652673
                                                                              
Generation 4 - Current best internal CV score: -3682.9075127652673
                                                                              
Generation 5 - Current best internal CV score: -3682.79982607396
                                                           
Best pipeline: LinearSVR(input_matrix, C=0.0001, dual=True, epsilon=0.01, loss=epsilon_insensitive, tol=0.01)
-3483.9933932198182


In [31]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)
tpot_350 = TPOTRegressor(generations=5, population_size=350, verbosity=2, cv=cv, random_state=42, n_jobs=-2)
tpot_350.fit(X_train, y_grs_train)
print(tpot_350.score(X_test, y_grs_test))
tpot_350.export('tpot_350_grs_pipeline.py')

tpot_350_gcs = TPOTRegressor(generations=5, population_size=350, verbosity=2, cv=cv, random_state=42, n_jobs=-2)
tpot_350_gcs.fit(X_gcs_train, y_gcs_train)
print(tpot_350_gcs.score(X_gcs_test, y_gcs_test))
tpot_350_gcs.export('tpot_350_gcs_pipeline.py')

                                                                                   
Generation 1 - Current best internal CV score: -165650662637.27472
                                                                                    
Generation 2 - Current best internal CV score: -162148449971.6318
                                                                                    
Generation 3 - Current best internal CV score: -160243932342.43372
                                                                                    
Generation 4 - Current best internal CV score: -157587980595.02185
                                                                                  
Generation 5 - Current best internal CV score: -153761057425.6203
                                                              
Best pipeline: ElasticNetCV(XGBRegressor(ZeroCount(input_matrix), learning_rate=0.5, max_depth=5, min_child_weight=4, n_estimators=100, n_jobs=1, objective=reg:squarederror, subsampl

In [4]:
X_dir  = list(df_working.iloc[:,0:1].columns)
y_grs = list(df_working.iloc[:,-1:].columns)
X_dir

['DIRECT_COST']

In [5]:
X_dir_train, X_dir_test, y_grs_train, y_grs_test = train_test_split(df_working[X_dir], df_working[y_grs].values.ravel(), test_size=0.2, random_state=42)

In [7]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)
tpot_dir_75_r2 = TPOTRegressor(generations=50, population_size=75, scoring='r2', verbosity=2, cv=cv, random_state=42, n_jobs=-2)
tpot_dir_75_r2.fit(X_dir_train, y_grs_train)
print(tpot_dir_75_r2.score(X_dir_test, y_grs_test))
tpot_dir_75_r2.export('tpot_dir_75_r2_grs_pipeline.py')

                                                                                 
Generation 1 - Current best internal CV score: 0.6924413345605032
                                                                                 
Generation 2 - Current best internal CV score: 0.6924413345605032
                                                                                 
Generation 3 - Current best internal CV score: 0.6924413345605032
                                                                                 
Generation 4 - Current best internal CV score: 0.6924413345605032
                                                                                 
Generation 5 - Current best internal CV score: 0.7051810612917506
                                                                                 
Generation 6 - Current best internal CV score: 0.7051810612917506
                                                                                 
Generation 7 - Current best in

c:\Users\SwanS\Anaconda3\envs\grs_fit\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but AdaBoostRegressor was fitted with feature names
  warnings.warn(
